# Import

In [18]:
!pip install -q googletrans
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import os
import re

import transformers
import tensorflow as tf
from tqdm.notebook import tqdm
from wordcloud import WordCloud, STOPWORDS
from sklearn.metrics import accuracy_score, roc_auc_score
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras import layers
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tokenizers import BertWordPieceTokenizer

from googletrans import Translator
from colorama import Fore, Back, Style, init
import plotly.graph_objects as go
translator = Translator()

from tensorflow.keras.layers import (Dense, Input, LSTM, Bidirectional, Activation, Conv1D, 
                                     GRU,Embedding, Flatten, Dropout, Add, concatenate, MaxPooling1D,
                                     GlobalAveragePooling1D,  GlobalMaxPooling1D, 
                                     GlobalMaxPool1D,SpatialDropout1D)

from tensorflow.keras import (initializers, regularizers, constraints, 
                              optimizers, layers, callbacks)

sns.set(style="darkgrid")

In [19]:
train_set1 = pd.read_csv(os.path.join( 'jigsaw-toxic-comment-train.csv'))
train_set2 = pd.read_csv(os.path.join( 'jigsaw-unintended-bias-train.csv'))
train_set2.toxic = train_set2.toxic.round().astype(int)

valid = pd.read_csv(os.path.join('validation.csv'))
test = pd.read_csv(os.path.join( 'test.csv'))

In [20]:
# Combine train1 with a subset of train2
train = pd.concat([
    train_set1[['comment_text', 'toxic']],
    train_set2[['comment_text', 'toxic']].query('toxic==1'),
    train_set2[['comment_text', 'toxic']].query('toxic==0').sample(n=100000, random_state=0)
])

# Modeling

#### < fast encoder >

In [21]:
def fast_encode(texts, tokenizer, chunk_size=240, maxlen=512):  # chunk_size 지정
    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding(max_length=maxlen)
    all_ids = []
    
    for i in range(0, len(texts), chunk_size):
        text_chunk = texts[i:i+chunk_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    
    return np.array(all_ids)

#### < general encoder >

In [22]:
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        return_attention_masks=False, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    return np.array(enc_di['input_ids'])

### TPU config

In [23]:
AUTO = tf.data.experimental.AUTOTUNE

# Create strategy from tpu
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)

# Configuration
EPOCHS = 4
BATCH_SIZE = 16* strategy.num_replicas_in_sync
MODEL = 'jplu/tf-xlm-roberta-large'
MAX_LEN = 224

ValueError: Please provide a TPU Name to connect to.

# 딥러닝; LSTM - RNN - NN(Neural Network) 

## - Neural Network

#### 인간의 신경체계를 모방한 정보처리 시스템 - 복잡, 비선형, 병렬계산
- 대량의 병렬 분산된 처리소자 -> 처리 노드가 많기 때문에 몇몇 노드나 연결에서의 결함이 시스템 전체의 심각한 결함을 초래하지는 않음
- 학습 알고리즘 -> 신경망의 출력값을 관찰하여 바람직한 출력값이 계산되도록 연결강도를 조정.  
- 분류 작업 수행
- 복잡한 비선형적 모델 추정하는데 유리 (비수렴성의 문제가 있긴 함)
- decision tree나 회귀와 같은 모델과는 달리 가중치의 해석/ 입력변수의 중요성 판단하기 어려움



### 분류 (구조)
- 단층 전방향 신경망
- 다층 전방향 신경망 --> RNN
- 순환 신경망
- 격자 신경망 (Self Organizing Map) : 비감시

![title](1.jpg)

#### 퍼셉트론 - 인공신경망의 시초로, 다수의 트레이닝 데이터를 기반으로 지도학습을 수행하는 알고리즘
- Xn : 입력값(데이터 특성을 나타내는 값)
- Wn : 가중치
- 순입력함수 : 특성값에 가중치를 곱한 값을 만드는 함수
- 활성함수 : 순입력함수의 결과값을 특정 임계값과 비교하여 이를 바탕으로 1 or -1 출력하는 함수

#### 즉, 위의 그림을 요약하면,
![title](2.jpg)

- 위의 그림과 같은 뉴런이 대량으로 연결되어 있다고 보면됨
- 입력층은 다른 노드의 출력값이 입력되는 것이고, 출력층은 어떤 노드의 출력값이 다른 노드로 전달되는 층이라고 보면됨
- 위와 같이 중간층이 하나의 노드로 구성되어 중간층과 출력층의 구분이 없는 구조를 단층 퍼셉트론이라함(중간층이 다수로 구성되어 있는 구조는 다층 퍼셉트론)

#### 다층 퍼셉트론 (MLP)
![title](3.jpg)

- 다층 퍼셉트론에서 입력층과 출력층 사이에 존재하는 층을 은닉층이라고 함
- 은닉층이 2개 이상인 신경망을 심층 신경망(DNN)이라고 함
- 이러한 다층 인공신경망을 학습하는 알고리즘을 딥러닝이라고 함


#### 다층 퍼셉트론 신경망은 입력이 출력방향으로만 활성화되고, 은닉 뉴런이 과거의 정보를 기억하지 못한다는 단점이 있음 -> RNN이 해결


## - RNN(Recurrent Neural Network)
#### 인공신경망 구조에서 순환 신경망으로 분류됨
<img src="4.jpg" width="400">
<img src="5.jpg" width="400">

- 히든노드(초록색 박스)가 방향을 가진 엣지로 연결되어 순환구조를 이루는 인공 신경망 -> 시퀀스 데이터의 모델링 가능
- '기억'을 갖고 있다는 특징이 있음(순전파) -> 지금까지의 입력데이터를 요약한 정보
- 즉, 새로운 입력이 들어올때마다 네크워크가 자신의 기억(이전 layer의 weight)을 조금씩 수정함(역전파) -> 입력을 모두 처리하면 남겨진 기억은 시퀀스 전체를 요약하는 정보
- Xt : 인풋, Ht : 히든(은닉) 노드, Yt : 출력값, Bn : 편향값
- 히든노드의 활성함수는 비선형함수인 tanh(역삼각함수)



### RNN이 할 수 있는 것?
1. 고정크기 입력 & 시퀀스 출력 : 이미지를 입력하면, 이미지의 캡션 자동 생성
2. 시퀀스 입력 & 고정크기 출력 : 문장을 입력해 긍/부정 정도를 출력하는 감정 분석기
3. 시퀀스 입력 & 시퀀스 출력 : 영어를 한국어로 번역하는 자동 번역기
4. 동기화된 시퀀스 입력 & 시퀀스 출력 : 문장에서 다음에 나올 단어 예측

#### RNN은 관련된 정보와 그 정보를 출력하는 지점의 사이가 멀어질수록 역전파시 gradient가 줄어들어 학습능력이 저하되는 단점(vanishing gradient problem)을 가지고 있음 --> LSTM이 해결

## - LSTM (Long Short Term Memory)
#### RNN의 hidden-state에 cell-state를 추가
<img src="6.jpg" width="500">


- cell-state는 4개의 특별한 상호작용을 함
- C는 기억/망각 정도를 학습한 셀
- forget gate는 이전의 state를 얼마나 잊을지 결정
- input gate는 현재 input으로 들어온 정보를 얼마나 기억할지 결정
- output gate는 얼마나 밖으로 표출할지 결정

<img src="7.jpg" width="400">
위의 gate들은 hidden unit과 x를 받아 활성함수로 sigmoid함수를 거쳐 0과 1사이 값을 출력함